# Sourcing Satelite Imagery from Google's Earth Engine

In [64]:
#Imports
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import shapely
import ee
from ee.data import exportMap
import folium
import io

# !pip install geemap
import geemap

# !brew install --cask wkhtmltopdf
# !pip install imgkit
import imgkit

from keys import thunderforest

from shapely.geometry import Point, Polygon, MultiPolygon, collection
import fiona
from shapely.ops import polygonize

In [65]:
#Authenticate Earth Engine
ee.Authenticate()

Enter verification code:  4/1AZEOvhXNmaTKlsZFqKNNl8M-CEfZDPcjYaivznyEfWu_VdVMc2pj8bQYMoY



Successfully saved authorization token.


In [66]:
ee.Initialize()

In [67]:
#Test API Connection
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


Cite: [Google's Earth Engine Colab Setup Guide](https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb)

## Select Projection and Imagery from Open Source Google Earth Engine

In [68]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = False,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
dem_vis_params = {
  'min': 0,
  'max': 1500,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[39.6, 19.83], width = 1000, height = 1000, zoom_start=10.5)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), dem_vis_params, 'DEM')
# my_map.add_ee_layer()

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Sourcing and Testing: Map Layers from [leaflet providers on Github](http://leaflet-extras.github.io/leaflet-providers/preview/)

In [69]:
#Thunderforest_MobileAtlas
folium.TileLayer()
my_map = folium.Map(location=[39.59, 19.83],
                    width = 800, height = 800,
                    zoom_start=10.5,
                    tiles = 'https://tile.thunderforest.com/mobile-atlas/{z}/{x}/{y}.png?'+'apikey='+thunderforest,
                    attr='<a href="http://www.thunderforest.com/">Thunderforest</a>, &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
                    min_zoom=10,
                    name='Thunderforest_MobileAtlas',overlay=True)

# var Thunderforest_MobileAtlas = L.tileLayer('https://{s}.tile.thunderforest.com/mobile-atlas/{z}/{x}/{y}.png?apikey={apikey}', {
	# attribution: '&copy; <a href="http://www.thunderforest.com/">Thunderforest</a>, &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
	# apikey: '<your apikey>',
	# maxZoom: 22
# });
# Display the map.
display(my_map)

In [70]:
#Google Maps Satelite Imagery
# cite: https://github.com/gee-community/geemap/blob/master/geemap/basemaps.py
folium.TileLayer()
my_map = folium.Map(location=[39.6, 19.83],
                    width = 800, height = 800,
                    zoom_start=10.3,
                    tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                    attr='Google',
                    name='Google Satelite')

# Display the map.
display(my_map)
my_map.save('../images/Corfu_Google_Satelite.html')

In [71]:
# ESRI World
my_map = folium.Map(location=[39.494, 19.873],
                    width = 500, height = 500,
                    zoom_start=18,
                    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                    attr='Tiles - Esri - Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',
                    min_zoom=10,
                    name='ESRI-World')

# Display the map.
display(my_map)
my_map.save('../images/Agios_Mattheos_ESRI_World.html')

In [72]:
#Google Maps Satelite Imagery
# cite: https://github.com/gee-community/geemap/blob/master/geemap/basemaps.py
folium.TileLayer()
my_map = folium.Map(location=[39.494, 19.873],
                    width = 500, height = 500,
                    zoom_start=18,
                    tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                    attr='Google',
                    name='Google Satelite', 
                    crs='EPSG3857'
                   )
print(my_map.crs)
# Display the map.
display(my_map)
my_map.save('../images/Agios_Mattheos_Google_Satelite.html')

EPSG3857


---
## Capture Small Scale Images

***Image Capture Process:***

1. Define Island Boundary via geojson or geographic polygon (see [02_Shoreline Boundary](code/02_Shoreline_Boundary.ipynb))
2. Initiate a grid to search over the island
3. If the starting geolocation is outside the geojson boundary, skip (this prevents hundreds of all blue images)
4. Capture image with name as lon_lat.png

In [73]:
# Read in Coastline Data
# Cite: https://geopandas.org/en/stable/docs/user_guide/io.html
coastline_shp = gpd.read_file('../data/clean_data/coastlines/corfu.shp')

In [74]:
type(coastline_shp)

geopandas.geodataframe.GeoDataFrame

In [75]:
coastline_shp.shape

(1, 2)

In [76]:
first_geom = coastline_shp.iloc[0]
coastline = gpd.GeoSeries(first_geom.geometry)

In [77]:
coastline

0    POLYGON ((19.68731 39.79651, 19.68751 39.79655...
dtype: geometry

In [78]:
# Cite: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.total_bounds.html
min_lon, min_lat, max_lon, max_lat = coastline.total_bounds

In [79]:
print(f"Westernmost Longitude: {min_lon}")
print(f"Easternmost Longitude: {max_lon}")
print(f"Southernmost Latitude: {min_lat}")
print(f"Northernmost Latitude: {max_lat}")

Westernmost Longitude: 19.629380550436906
Easternmost Longitude: 20.12227358704468
Southernmost Latitude: 39.36066892455921
Northernmost Latitude: 39.822028253950144


In [80]:
#Estimate the number of images needed to cover the island at the set offsets
print(f"Vertical: {(max_lat - min_lat) / 0.0012}")
print(f"Horizontal: {(max_lon - min_lon) / 0.0016}")

print(f"{round(((max_lat - min_lat) / 0.0012) * (max_lon - min_lon) / 0.0016)} Images Per Grid")

Vertical: 384.46610782578125
Horizontal: 308.0581478798594
118438 Images Per Grid


## Down-Select for a Smaller Footprint

Here the Municipality of Agros is leveraged to down-scale the images captured.

Higher capacity models could be leveraged to evaluate larger geographic areas by adjusting the border.

In [81]:
muni_shp = gpd.read_file('../data/clean_data/coastlines/agros.shp')

In [82]:
type(muni_shp)

geopandas.geodataframe.GeoDataFrame

In [83]:
first_geom = muni_shp.iloc[0]
muni = gpd.GeoSeries(first_geom.geometry)

In [84]:
#Extract Agros
muni

0    MULTIPOLYGON (((19.63613 39.72388, 19.63579 39...
dtype: geometry

In [85]:
point = shapely.geometry.Point(19.686689, 39.722445)
muni.contains(point).item()

True

## Capture Images Gridwise within the set boundaries

In [86]:
lat = 39.596
lon = 19.8015
m = folium.Map(location=[lat, lon],
                    width = 300, height = 310,
                    min_zoom=18, max_zoom = 18,
                    tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                    attr='Google',
                    name='Google Satelite',
                    disable_3d=True,
                    control_scale=False,
                   zoom_control = False,
                   scrollWheelZoom = False,
                   dragging = False)

# Display the map.
display(m)

#Save the Image - Folium Default is html
#cite: https://python-visualization.github.io/folium/plugins.html?highlight=save
# m.save('image.html')

# imgkit approach
#cite: Mohit Chandel https://stackoverflow.com/a/60598918

## Save as html file to temp folder
m.save(f'../images/temp/{lon}_{lat}_sat.html')

## Convert to png file and save to permanent folder
imgkit.from_file(f'../images/temp/{lon}_{lat}_sat.html',f'../images/satelite_tiles/{lon}_{lat}_sat.png')

##Destroy html file 
os.remove(f'../images/temp/{lon}_{lat}_sat.html')

OSError: wkhtmltoimage reported an error:
Loading page (1/2)
[>                                                           ] 0%[======>                                                     ] 10%[======>                                                     ] 11%[===========>                                                ] 19%[==============>                                             ] 24%[================>                                           ] 28%[===================>                                        ] 32%[======================>                                     ] 37%[=========================>                                  ] 42%[===========================>                                ] 46%[============================>                               ] 48%[================================>                           ] 54%[=================================>                          ] 56%[=======================================>                    ] 65%[=

In [ ]:
# Function Approach: Plot Map, Check for Boundaries, Save .png file over a set grid.

In [ ]:
def create_map_tiles(lati, long, border, width, height):
    point = shapely.geometry.Point(long, lati)
    #Continue if the selected point does not lie within the border geometry
    if border.contains(point).item():
        #Generate the map with Folium
        m = folium.Map(location=[lati, long],
                    width = width, height = height,
                    min_zoom=18, max_zoom = 18,
                    tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                    attr='Google',
                    name='Google Satelite',
                    disable_3d=True,
                    control_scale=False,
                    zoom_control = False,
                    scrollWheelZoom = False,
                    dragging = False)
        # display(m)
        
        # Save the map as an html from folium
        m.save(f'../images/temp/{lati}_{long}_sat.html')
        ## Convert to png file and save to permanent folder
        imgkit.from_file(f'../images/temp/{lati}_{long}_sat.html',f'../data/clean_data/satelite_images/{lati}_{long}_sat.png')
        ##Destroy html file 
        os.remove(f'../images/temp/{lati}_{long}_sat.html') #regex to read in lonlat from filepath
        return
    else:
        return

In [ ]:
#Single Tile Test
create_map_tiles(39.722446, 19.686688, muni, 300 , 300)

In [87]:
min_lon, min_lat, max_lon, max_lat = muni.total_bounds
min_lat = round(min_lat,5)
min_lon = round(min_lon,5)
max_lat = round(max_lat,5)
max_lon = round(max_lon,5)
print(min_lat, min_lon, max_lat, max_lon)

39.69354 19.63433 39.75911 19.75314


In [88]:
#Estimate the number of images needed to cover the island at the set offsets
print(f"Vertical: {(max_lat - min_lat) / 0.0012}")
print(f"Horizontal: {(max_lon - min_lon) / 0.0016}")

print(f"{round(((max_lat - min_lat) / 0.0012) * (max_lon - min_lon) / 0.0016)} Images Per Grid")

Vertical: 54.64166666666752
Horizontal: 74.25624999999991
4057 Images Per Grid


In [89]:
#Leverage the movable maps to select the change in latitude and longitude you want to move between capturing images.
border = muni
lon_interval = 0.0016
width = 300
lat_interval = 0.0012
height = 300


#plot a boundary using the northernmost, easternmost, etc. points in your border geometry
# min_lon, min_lat, max_lon, max_lat = coastline.total_bounds

for y in np.arange(min_lat, max_lat, lat_interval):
    for x in np.arange(min_lon, max_lon, lon_interval):
        create_map_tiles(y, x, border, width, height)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                    

OSError: wkhtmltoimage reported an error:
Loading page (1/2)
[>                                                           ] 0%[======>                                                     ] 10%[======>                                                     ] 11%[===========>                                                ] 19%[===============>                                            ] 26%[================>                                           ] 28%[===================>                                        ] 32%[======================>                                     ] 37%[========================>                                   ] 41%[==========================>                                 ] 44%Error: Failed to load https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js, with network status code 99 and http status code 0 - Operation timed out
Error: Failed to load https://code.jquery.com/jquery-1.12.4.min.js, with netwo

In [ ]:
# Resample using a shifted matrix by one half the window height and width.  
#This should eliminate instances where houses land on image boundaries.
border = muni
lon_interval = 0.0016
width = 300
lat_interval = 0.0012
height = 300


#plot a boundary using the northernmost, easternmost, etc. points in your border geometry
min_lon, min_lat, max_lon, max_lat = coastline.total_bounds

for x in np.arange(min_lat + 0.5 * lat_interval,
                   max_lat - 0.5 * lat_interval,
                   lat_interval):
    for y in np.arange(min_lon + 0.5 * lon_interval,
                       max_lon - 0.5 * lon_interval,
                       lon_interval):
        create_map_tiles(x, y, border, width, height)

> Result: Images covering the full island with minimal inclusion of the sea, captured at half-frame intervals vertically and horizontally.